In [1]:
from dotenv import dotenv_values
import requests

from helpers.google_interactor import GoogleInteractor
from helpers.html_parser import HTMLParser 
from helpers.storage_manager import StorageManager
from helpers.refiner_model import RefinerModel
from helpers.improver_model import ImproverModel
from helpers.writer_citer_model import WriterCiterModel


"""
============================================
Run through a single trial of the experiment
============================================
"""

"""
Set up
"""

# Number of links to consider
K = 5

# Declare trial number
trial_number = 13

# Select a query to use for the current trial
query = "What happens if I outlive my term life insurance policy?"

"""
Configuration
"""

sfblic_link = "https://www.sfbli.com/"

# Load environment variables
config = dotenv_values("../.env")
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

# Instantiate helpers
google_interactor = GoogleInteractor(api_key=google_api_key, search_engine_id=search_engine_id)
storage_manager = StorageManager("../data/")    
website_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
corpus_refiner = RefinerModel(api_key=config['OPEN_AI_API_KEY'])
improver_model = ImproverModel(api_key=config['OPEN_AI_API_KEY'])
writer_citer = WriterCiterModel(api_key=config['OPEN_AI_API_KEY'])


# Load the clean corpus text
with open("../data/clean_content.txt") as f:
    corpus_text = f.read()

In [2]:
"""
Submit the query to Google and store the results
"""

# Submit the query to Google
search_results = google_interactor.search_google(query=query, num_results=10)
    
# Store the response
storage_manager.save_to_folder("InitialGoogleResponses", search_results, f"{trial_number}.json")

Data saved to ../data/InitialGoogleResponses/13.json


In [3]:
"""
View links
"""

top_k_links = [item['link'] for item in search_results['items']][:K]
top_k_links

['https://www.bankrate.com/insurance/life-insurance/outlive-term-life-insurance/',
 'https://www.aflac.com/resources/life-insurance/what-happens-when-a-term-life-insurance-policy-expires.aspx',
 'https://www.policyadvisor.com/life-insurance/outlive-term-life-insurance/',
 'https://www.investopedia.com/what-happens-if-you-outlive-your-term-life-insurance-policy-5076607',
 'https://www.quotacy.com/what-happens-if-i-dont-die-while-my-policy-is-inforce/']

In [4]:
"""
Check to see if SFBLIC is in the top K links. 
If not, add it
"""

if sfblic_link in top_k_links:
    updated_links = top_k_links
else:
    top_k_links[K-1] = sfblic_link
    updated_links = top_k_links

updated_links

['https://www.bankrate.com/insurance/life-insurance/outlive-term-life-insurance/',
 'https://www.aflac.com/resources/life-insurance/what-happens-when-a-term-life-insurance-policy-expires.aspx',
 'https://www.policyadvisor.com/life-insurance/outlive-term-life-insurance/',
 'https://www.investopedia.com/what-happens-if-you-outlive-your-term-life-insurance-policy-5076607',
 'https://www.sfbli.com/']

In [5]:
storage_manager.save_to_folder("UpdatedGoogleResponses", updated_links, f"{trial_number}.json")

Data saved to ../data/UpdatedGoogleResponses/13.json


In [6]:
"""
Get Raw Text from the HTML associated with the links from Google
"""

# Retrieve the stored response
updated_links = storage_manager.load_from_folder("UpdatedGoogleResponses", f"{trial_number}.json")

raw_text_from_html = []

# For each item of raw text that was stored ...
for link in updated_links:    

    try:

        # Fet the HTML associated with the link
        response = requests.get(link)
    
        # Get the raw text associated with the HTML
        parsed_text = HTMLParser.parse_html(response.content)
    
        # Store the raw text from the HTML
        raw_text_from_html.append({"link" : link, "raw_text" : parsed_text})

    except:
        print("Something happened with ", link)
    
storage_manager.save_to_folder("RawTextFromHTML", raw_text_from_html, f"{trial_number}.json")

Data loaded from ../data/UpdatedGoogleResponses/13.json
Data saved to ../data/RawTextFromHTML/13.json


In [7]:
link

'https://www.sfbli.com/'

In [8]:
"""
Get the refined text from the raw text
"""

raw_text_from_html = storage_manager.load_from_folder("RawTextFromHTML", f"{trial_number}.json")

website_refiner_responses = []
# For each element of raw text that was stored ...
for current_item in raw_text_from_html:
    
    link = current_item['link']
    raw_text = current_item['raw_text']   

    
    response_from_website_refiner = website_refiner.refine_text(raw_text=raw_text, query=query)
    
    if response_from_website_refiner:
        response_from_website_refiner['link'] = link
        website_refiner_responses.append(response_from_website_refiner)
    else:
        print("Failed to refine text.")

storage_manager.save_to_folder("WebsiteRefinerModelResponses", website_refiner_responses, f"{trial_number}.json")

Data loaded from ../data/RawTextFromHTML/13.json
Data saved to ../data/WebsiteRefinerModelResponses/13.json


In [9]:
for item in website_refiner_responses:
    print(item['link'])
    print(item['choices'][0]['message']['content'])
    print()

https://www.bankrate.com/insurance/life-insurance/outlive-term-life-insurance/
When term life insurance expires, it typically ends without any action needed from the policyholder. The insurance carrier sends a notice, premiums stop, and there is no longer a death benefit. If the policy included a return-of-premium feature, the policyholder would receive a check for the premiums paid during the term.

Some term policies offer the option to renew on a year-by-year basis after the initial term expires, which maintains the original coverage amount but is usually more expensive each year due to age-related risk increases. This option is beneficial for those who develop health issues that make obtaining new insurance challenging.

Additionally, many term policies include a conversion rider that allows the policyholder to convert their term policy into a permanent policy without needing to go through underwriting again. This is a strategic choice for those who develop a health condition or be

In [10]:
"""
Filter the corpus to information that may be relevant to the query
"""

response_from_corpus_refiner = corpus_refiner.refine_text(raw_text=corpus_text, query=query)
storage_manager.save_to_folder("CorpusRefinerModelResponses", response_from_corpus_refiner, f"{trial_number}.json")

Data saved to ../data/CorpusRefinerModelResponses/13.json


In [11]:
print(response_from_corpus_refiner['choices'][0]['message']['content'])

When term life insurance expires, it typically ends without any action needed from the policyholder. The insurance carrier sends a notice, premiums stop, and there is no longer a death benefit. If the policy had a return-of-premium feature, the policyholder might receive a check for the premiums paid during the term.

Some term policies offer the option to renew on a year-by-year basis after the initial term expires, maintaining the original coverage amount but usually at a higher cost due to age-related risk increases. Additionally, many term policies include a conversion rider that allows the policyholder to convert the term policy into a permanent policy without going through underwriting again, although this option may have specific time restrictions.

If coverage is still needed after the term expires, individuals can evaluate their options, such as converting their policy if they have a conversion rider, renewing their term policy, buying a new term policy, or purchasing a perman

In [12]:
"""
Improve the text from the SFBLIC site
"""

# Get the refined text from the website
website_refiner_responses = storage_manager.load_from_folder("WebsiteRefinerModelResponses", f"{trial_number}.json")
sfblic_response = [response for response in website_refiner_responses if response['link'] == 'https://www.sfbli.com/'][0]
refined_text_from_website = sfblic_response['choices'][0]['message']['content']

# Get the refined text from the corpus
response_from_corpus_refiner = storage_manager.load_from_folder("CorpusRefinerModelResponses", f"{trial_number}.json")
refined_text_from_corpus = response_from_corpus_refiner['choices'][0]['message']['content']


# Generate improved text
improved_text = improver_model.improve_text(
    query=query,
    refined_text_from_website=refined_text_from_website,
    refined_text_from_corpus=refined_text_from_corpus
)

if improved_text:
    print("Improved Text generated...")
    
else:
    print("Failed to improve text.")


# Save to disk
storage_manager.save_to_folder("ImproverModelResponses", improved_text, f"{trial_number}.json")


Data loaded from ../data/WebsiteRefinerModelResponses/13.json
Data loaded from ../data/CorpusRefinerModelResponses/13.json
Improved Text generated...
Data saved to ../data/ImproverModelResponses/13.json


In [13]:
print(refined_text_from_website)

THERE IS NO RELEVANT INFORMATION IN THE RAW TEXT


In [14]:
print(improved_text['choices'][0]['message']['content'])

If you outlive your term life insurance policy, the policy typically expires without any action needed from you. You will no longer have life insurance coverage, and there will be no death benefit paid out. If your policy had a return-of-premium feature, you might receive a refund of the premiums you paid over the term.

After the policy expires, you have a few options if you still need coverage. Some term policies allow you to renew them on a year-by-year basis, but usually at a higher cost due to increased age-related risks. Alternatively, if your policy includes a conversion rider, you may convert your term policy into a permanent one without undergoing the underwriting process again, though there may be time restrictions for this option.

If you determine that you still need life insurance coverage, you can evaluate whether to convert your existing policy, renew the term policy, buy a new term policy, or consider a permanent life insurance policy. Permanent policies tend to be more

In [15]:
"""
Create multiple sets of reference material
"""

# Create a set of reference material that has the original (unimproved) content from the website

website_refiner_model_responses = storage_manager.load_from_folder("WebsiteRefinerModelResponses", f"{trial_number}.json")

unmodified_set_of_reference_material =  [
    {
        "link" : item['link'],
        "information" : item['choices'][0]['message']['content']
    }
    for item in website_refiner_model_responses
]

# Create a set of reference material that has the new (improved) content
improved_text = storage_manager.load_from_folder('ImproverModelResponses', f"{trial_number}.json")
improved_version = improved_text['choices'][0]['message']['content']


# Create a new set of reference material

new_set_of_reference_material = []

for item in unmodified_set_of_reference_material:
    if item['link'] == 'https://www.sfbli.com/':
        new_set_of_reference_material.append({"link" : link, "information" : improved_version})
    else:
        new_set_of_reference_material.append(item)

Data loaded from ../data/WebsiteRefinerModelResponses/13.json
Data loaded from ../data/ImproverModelResponses/13.json


In [16]:
"""
Store Sets of Reference Material
"""

sets_of_reference_material = []

improvement_method = 0
set_of_reference_material = unmodified_set_of_reference_material

sets_of_reference_material.append({
    "improvement_method" : improvement_method,
    "set_of_reference_material" : set_of_reference_material,
})

improvement_method = 1
set_of_reference_material = new_set_of_reference_material

sets_of_reference_material.append({
    "improvement_method" : improvement_method,
    "set_of_reference_material" : set_of_reference_material,
})


storage_manager.save_to_folder("SetsOfReferenceMaterial", sets_of_reference_material, f"{trial_number}.json")

Data saved to ../data/SetsOfReferenceMaterial/13.json


In [17]:
"""
Send sets of reference material to the writer-citer
"""

writer_citer_model_responses = [] 

sets_of_reference_material = storage_manager.load_from_folder("SetsOfReferenceMaterial", f"{trial_number}.json")

# Unimproved Method
# -----------------

set_of_reference_material = sets_of_reference_material[0]
improvement_method = set_of_reference_material['improvement_method']
reference_material = set_of_reference_material['set_of_reference_material']

# Generate a cited response
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=reference_material
)

cited_response['improvement_method'] = improvement_method

writer_citer_model_responses.append(cited_response)

# Improved Method
# -----------------

set_of_reference_material = sets_of_reference_material[1]
improvement_method = set_of_reference_material['improvement_method']
reference_material = set_of_reference_material['set_of_reference_material']

# Generate a cited response
cited_response = writer_citer.generate_cited_response(
    rephrased_query=query,
    set_of_reference_query=reference_material
)

cited_response['improvement_method'] = improvement_method

writer_citer_model_responses.append(cited_response)


Data loaded from ../data/SetsOfReferenceMaterial/13.json


In [18]:
for item in writer_citer_model_responses:
    print(item['choices'][0]['message']['content'])
    print("---")

If you outlive your term life insurance policy, the coverage ends, and you stop paying premiums, resulting in no death benefit being available for your beneficiaries [1][2]. However, several options may be available depending on your needs and circumstances:

1. **Renew Your Policy**: Many insurers offer the possibility to renew the term on a year-by-year basis, but the premiums generally increase due to the higher risk associated with aging [1][3].

2. **Convert Your Policy**: Some term policies allow you to convert them into permanent policies, such as whole or universal life insurance, without requiring a new medical exam. This can be advantageous if your health has deteriorated, making it difficult to qualify for new insurance [1][2][3].

3. **Buy a New Policy**: If you still require life insurance coverage, purchasing a new policy is an option. This typically involves higher premiums and possibly a medical exam, especially if you are older or seek substantial coverage. In some cas

In [19]:
# Save to disk
storage_manager.save_to_folder("WriterCiterModelResponses", writer_citer_model_responses, f"{trial_number}.json")

Data saved to ../data/WriterCiterModelResponses/13.json


In [20]:
print("hello")

hello
